**Word Sense Disambiguation**

**[
[word1, [sentence1, sentence2 ...], description, postag] ,
[word2, .....................................................................],
.
.
.
.
]**

In [1]:
from trLemmer import MorphAnalyzer
lemmatizer = MorphAnalyzer()
wsd = []

In [2]:
def create(word, description, postag):
    wsd.append([word, [], postag, description])

In [3]:
def add(sent, word, postag):
    isThereOnList = False
    indexCount = 0
    for i in wsd:
        if i[0] == word and i[2] == postag:
            isThereOnList = True
            break
        else:
            indexCount = indexCount + 1
    if isThereOnList:
        wsd[indexCount][1].append(sent)
    else:
        return "Böyle bir kelime yok!"

In [4]:
def examples(word):
    indexCount = 0
    for i in wsd:
        if i[0] == word:
            print("Examples of", word, "(", i[2], ")", ": \n", wsd[indexCount][1])
        indexCount = indexCount + 1            

In [5]:
def description(word):
    postagCount = []
    isThereOnList = False
    indexCount = 0
    for i in wsd:
        if i[0] == word:
            for j in postagCount:
                if j != i[2]:
                    print("Description of", word, "(", i[2], ")", ": \n",  wsd[indexCount][3])
                    postagCount.append(i[2])
        indexCount = indexCount + 1       

In [6]:
def lesk(words, index):
    lemmatizer = MorphAnalyzer()
    distanceWeight = []
    distancePostag = []
    for l in words:
        word = lemmatizer.lemmatize(l)[0]
        for i in wsd:
            if i[0] == str(word):
                for j in i[1]:
                    for k in j:
                        distanceWeight.append([levenshteinDistanceDP(lemmatizer.lemmatize(k)[0], lemmatizer.lemmatize(words[index])[0], False), i[2]])
            similar = i
            break
    minDistance = [100, ""]
    for i in distanceWeight:
        if i[0] < minDistance[0]:
            minDistance = i
    for i in distanceWeight:
        if i[0] == minDistance[0]:
            distancePostag.append(i[1])
    for i in wsd:
        if i[0] == str(lemmatizer.lemmatize(words[index])[0]) and i[2] == minDistance[1]:
            description = i[3]
    return lemmatizer.lemmatize(words[index])[0], minDistance, "-->", description

In [7]:
def printDistances(distances, token1Length, token2Length):
    for t1 in range(token1Length + 1):
        for t2 in range(token2Length + 1):
            print(int(distances[t1][t2]), end=" ")
        print()

In [8]:
def levenshteinDistanceDP(token1, token2, print = True):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    if print:
        printDistances(distances, len(token1), len(token2))
    return distances[len(token1)][len(token2)]

In [9]:
create('metin', 'Bir yazıyı biçim, anlatım ve noktalama özellikleriyle oluşturan kelimelerin bütünü, tekst', 'isim')

In [10]:
add(['Topladığın', 'metinleri', 'kağıda', 'yaz', '.'], 'metin', 'isim')

In [11]:
wsd

[['metin',
  [['Topladığın', 'metinleri', 'kağıda', 'yaz', '.']],
  'isim',
  'Bir yazıyı biçim, anlatım ve noktalama özellikleriyle oluşturan kelimelerin bütünü, tekst']]

In [12]:
add(['Kendisi', 'çok', 'metin', 'bir', 'insan', '.'], 'metin', 'sıfat')

'Böyle bir kelime yok!'

In [13]:
create('metin', 'acılara dayanabilen, dayanıklı kimse', 'sıfat')

In [14]:
wsd

[['metin',
  [['Topladığın', 'metinleri', 'kağıda', 'yaz', '.']],
  'isim',
  'Bir yazıyı biçim, anlatım ve noktalama özellikleriyle oluşturan kelimelerin bütünü, tekst'],
 ['metin', [], 'sıfat', 'acılara dayanabilen, dayanıklı kimse']]

In [15]:
add(['Kendisi', 'çok', 'metin', 'bir', 'insan', '.'], 'metin', 'sıfat')

In [16]:
wsd

[['metin',
  [['Topladığın', 'metinleri', 'kağıda', 'yaz', '.']],
  'isim',
  'Bir yazıyı biçim, anlatım ve noktalama özellikleriyle oluşturan kelimelerin bütünü, tekst'],
 ['metin',
  [['Kendisi', 'çok', 'metin', 'bir', 'insan', '.']],
  'sıfat',
  'acılara dayanabilen, dayanıklı kimse']]

In [17]:
create('yaz', 'Kuzey yarım kürede 21 Haziran 23 Eylül tarihleri arasındaki zaman dilimi, ilkbaharla sonbahar arasındaki sıcak mevsim', 'isim')
create('yaz', 'Nakşetmek, resmetmek, süsleyip bezemek.', 'fiil')

In [18]:
add(['Yaz', 'sıcaklarını', 'hiç', 'sevmem', '.'], 'yaz', 'isim')

In [19]:
wsd

[['metin',
  [['Topladığın', 'metinleri', 'kağıda', 'yaz', '.']],
  'isim',
  'Bir yazıyı biçim, anlatım ve noktalama özellikleriyle oluşturan kelimelerin bütünü, tekst'],
 ['metin',
  [['Kendisi', 'çok', 'metin', 'bir', 'insan', '.']],
  'sıfat',
  'acılara dayanabilen, dayanıklı kimse'],
 ['yaz',
  [['Yaz', 'sıcaklarını', 'hiç', 'sevmem', '.']],
  'isim',
  'Kuzey yarım kürede 21 Haziran 23 Eylül tarihleri arasındaki zaman dilimi, ilkbaharla sonbahar arasındaki sıcak mevsim'],
 ['yaz', [], 'fiil', 'Nakşetmek, resmetmek, süsleyip bezemek.']]

In [20]:
add(['Sunumda', 'geçen', 'konuları', 'yaz', '.'], 'yaz', 'fiil')

In [21]:
add(['Metinde', 'geçen', 'bilgileri', 'yazdım', '.'], 'yaz', 'fiil')

In [22]:
wsd

[['metin',
  [['Topladığın', 'metinleri', 'kağıda', 'yaz', '.']],
  'isim',
  'Bir yazıyı biçim, anlatım ve noktalama özellikleriyle oluşturan kelimelerin bütünü, tekst'],
 ['metin',
  [['Kendisi', 'çok', 'metin', 'bir', 'insan', '.']],
  'sıfat',
  'acılara dayanabilen, dayanıklı kimse'],
 ['yaz',
  [['Yaz', 'sıcaklarını', 'hiç', 'sevmem', '.']],
  'isim',
  'Kuzey yarım kürede 21 Haziran 23 Eylül tarihleri arasındaki zaman dilimi, ilkbaharla sonbahar arasındaki sıcak mevsim'],
 ['yaz',
  [['Sunumda', 'geçen', 'konuları', 'yaz', '.'],
   ['Metinde', 'geçen', 'bilgileri', 'yazdım', '.']],
  'fiil',
  'Nakşetmek, resmetmek, süsleyip bezemek.']]

In [23]:
examples('yaz')

Examples of yaz ( isim ) : 
 [['Yaz', 'sıcaklarını', 'hiç', 'sevmem', '.']]
Examples of yaz ( fiil ) : 
 [['Sunumda', 'geçen', 'konuları', 'yaz', '.'], ['Metinde', 'geçen', 'bilgileri', 'yazdım', '.']]


In [24]:
examples('metin')

Examples of metin ( isim ) : 
 [['Topladığın', 'metinleri', 'kağıda', 'yaz', '.']]
Examples of metin ( sıfat ) : 
 [['Kendisi', 'çok', 'metin', 'bir', 'insan', '.']]


In [25]:
for i in wsd[3][1][1]:
    print(lemmatizer.lemmatize(i))

['metîn', 'metin']
['geçmek', 'geçen']
['bilgi']
['yazmak', 'yaz']
['.']


In [26]:
print(wsd[0][1][0])
for i in wsd[0][1]:
    for j in i:
        print(lemmatizer.lemmatize(j)[0])

['Topladığın', 'metinleri', 'kağıda', 'yaz', '.']
toplamak
metîn
kâğıt
yazmak
.


In [27]:
print(wsd[3][1][1])
for i in wsd[3][1][1]:
    print(lemmatizer.lemmatize(i)[0])

['Metinde', 'geçen', 'bilgileri', 'yazdım', '.']
metîn
geçmek
bilgi
yazmak
.


In [42]:
levenshteinDistanceDP("asla", "asla")

0 1 2 3 4 
1 0 1 2 3 
2 1 0 1 2 
3 2 1 0 1 
4 3 2 1 0 


0.0

In [ ]:
for i in wsd[0][1][0]:
    print(lemmatizer.lemmatize(i)[0], "x", "yaz")
    print(levenshteinDistanceDP(lemmatizer.lemmatize(i)[0], "yaz"))

In [ ]:
lesk(["Verdiğim", "metinlere", "iyi", "bak"], 1)